## Shapefiles

In [1]:
import geopandas

In [2]:
df = geopandas.read_file("global_tectonics/polygon_data/GSHHS_I_L1.shp")

In [3]:
df = df[~geopandas.pd.isnull(df.id)]

In [4]:
df = df[df.area > 100]

/tmp/ipykernel_62108/3268482086.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  df = df[df.area > 100]


In [5]:
geometries = df["geometry"]

In [6]:
from pyproj import Proj
to_stereographic = Proj("+proj=stere +lat_0=90 +R=1.0 +lat_ts=0")

In [7]:
import numpy as np

def create_surface_from_geometry(geometry):

    coords = list(geometry.exterior.coords)
    coords_lon_lat_degrees = np.array(coords)
    
    x, y = to_stereographic(coords_lon_lat_degrees[:,0], 
                            coords_lon_lat_degrees[:,1])
    
    coords_stereographic = np.zeros_like(coords)
    coords_stereographic[:, 0] = x
    coords_stereographic[:, 1] = y
    
    ## Add points
    
    point_tags = []

    # Here we account for the fact that, in a polygon shapefile, the first and
    # last node of a polygon are the same.
    for index in range(len(coords_stereographic) - 1):
        x = coords_stereographic[index, 0]
        y = coords_stereographic[index, 1]
        tag = gmsh.model.geo.add_point_on_geometry(sphere, x, y)
        point_tags.append(tag)
        
    point_tags = point_tags + [point_tags[0]]
    
    ## Add lines
    
    line_tags = []

    for index in range(len(point_tags) - 1):
        pt1, pt2 = point_tags[index], point_tags[index + 1]
        tag = gmsh.model.geo.add_line(pt1, pt2)
        line_tags.append(tag)
        
    cl1 = gmsh.model.geo.add_curve_loop(line_tags)
    gmsh.model.geo.add_plane_surface([cl1])

## GMSH Polar Sphere

In [8]:
import gmsh

In [9]:
gmsh.initialize()

In [10]:
gmsh.model.add("Earth")

In [11]:
sphere = gmsh.model.geo.add_geometry('PolarSphere', [0., 0., 0., 1.0])

In [12]:
for geometry in geometries:
    create_surface_from_geometry(geometry)

In [13]:
gmsh.model.geo.synchronize()

Info    : SVD failed (angle=84.9321): using rough algo...
Info    : SVD failed (angle=91.3616): using rough algo...
Info    : SVD failed (angle=89.5014): using rough algo...


### Adapt the mesh

In [14]:
gmsh.model.mesh.field.add("Distance", 1)
gmsh.model.mesh.field.setNumbers(1, "CurvesList", [d[1] for d in gmsh.model.getEntities(1)])


gmsh.model.mesh.field.add("Threshold", 2)
gmsh.model.mesh.field.setNumber(2, "InField", 1)
gmsh.model.mesh.field.setNumber(2, "SizeMin", 0.01)
gmsh.model.mesh.field.setNumber(2, "SizeMax", 0.1)
gmsh.model.mesh.field.setNumber(2, "DistMin", 0.01)
gmsh.model.mesh.field.setNumber(2, "DistMax", 0.2)

gmsh.model.mesh.field.setAsBackgroundMesh(2)

In [15]:
gmsh.option.setNumber("Mesh.MeshSizeExtendFromBoundary", 0)

In [16]:
gmsh.model.mesh.generate(2)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    : [ 10%] Meshing curve 19 (Line)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Line)
Info    : [ 10%] Meshing curve 22 (Line)
Info    : [ 10%] Meshing curve 23 (Line)
Info    : [ 10%] Meshing curve 24 (Line)
I

Info    : [ 10%] Meshing curve 368 (Line)
Info    : [ 10%] Meshing curve 369 (Line)
Info    : [ 10%] Meshing curve 370 (Line)
Info    : [ 10%] Meshing curve 371 (Line)
Info    : [ 10%] Meshing curve 372 (Line)
Info    : [ 10%] Meshing curve 373 (Line)
Info    : [ 10%] Meshing curve 374 (Line)
Info    : [ 10%] Meshing curve 375 (Line)
Info    : [ 10%] Meshing curve 376 (Line)
Info    : [ 10%] Meshing curve 377 (Line)
Info    : [ 10%] Meshing curve 378 (Line)
Info    : [ 10%] Meshing curve 379 (Line)
Info    : [ 10%] Meshing curve 380 (Line)
Info    : [ 10%] Meshing curve 381 (Line)
Info    : [ 10%] Meshing curve 382 (Line)
Info    : [ 10%] Meshing curve 383 (Line)
Info    : [ 10%] Meshing curve 384 (Line)
Info    : [ 10%] Meshing curve 385 (Line)
Info    : [ 10%] Meshing curve 386 (Line)
Info    : [ 10%] Meshing curve 387 (Line)
Info    : [ 10%] Meshing curve 388 (Line)
Info    : [ 10%] Meshing curve 389 (Line)
Info    : [ 10%] Meshing curve 390 (Line)
Info    : [ 10%] Meshing curve 391

Info    : [ 20%] Meshing curve 2526 (Line)
Info    : [ 20%] Meshing curve 2527 (Line)
Info    : [ 20%] Meshing curve 2528 (Line)
Info    : [ 20%] Meshing curve 2529 (Line)
Info    : [ 20%] Meshing curve 2530 (Line)
Info    : [ 20%] Meshing curve 2531 (Line)
Info    : [ 20%] Meshing curve 2532 (Line)
Info    : [ 20%] Meshing curve 2533 (Line)
Info    : [ 20%] Meshing curve 2534 (Line)
Info    : [ 20%] Meshing curve 2535 (Line)
Info    : [ 20%] Meshing curve 2536 (Line)
Info    : [ 20%] Meshing curve 2537 (Line)
Info    : [ 20%] Meshing curve 2538 (Line)
Info    : [ 20%] Meshing curve 2539 (Line)
Info    : [ 20%] Meshing curve 2540 (Line)
Info    : [ 20%] Meshing curve 2541 (Line)
Info    : [ 20%] Meshing curve 2542 (Line)
Info    : [ 20%] Meshing curve 2543 (Line)
Info    : [ 20%] Meshing curve 2544 (Line)
Info    : [ 20%] Meshing curve 2545 (Line)
Info    : [ 20%] Meshing curve 2546 (Line)
Info    : [ 20%] Meshing curve 2547 (Line)
Info    : [ 20%] Meshing curve 2548 (Line)
Info    : [

Info    : [ 20%] Meshing curve 2910 (Line)
Info    : [ 20%] Meshing curve 2911 (Line)
Info    : [ 20%] Meshing curve 2912 (Line)
Info    : [ 20%] Meshing curve 2913 (Line)
Info    : [ 20%] Meshing curve 2914 (Line)
Info    : [ 20%] Meshing curve 2915 (Line)
Info    : [ 20%] Meshing curve 2916 (Line)
Info    : [ 20%] Meshing curve 2917 (Line)
Info    : [ 20%] Meshing curve 2918 (Line)
Info    : [ 20%] Meshing curve 2919 (Line)
Info    : [ 20%] Meshing curve 2920 (Line)
Info    : [ 20%] Meshing curve 2921 (Line)
Info    : [ 20%] Meshing curve 2922 (Line)
Info    : [ 20%] Meshing curve 2923 (Line)
Info    : [ 20%] Meshing curve 2924 (Line)
Info    : [ 20%] Meshing curve 2925 (Line)
Info    : [ 20%] Meshing curve 2926 (Line)
Info    : [ 20%] Meshing curve 2927 (Line)
Info    : [ 20%] Meshing curve 2928 (Line)
Info    : [ 20%] Meshing curve 2929 (Line)
Info    : [ 20%] Meshing curve 2930 (Line)
Info    : [ 20%] Meshing curve 2931 (Line)
Info    : [ 20%] Meshing curve 2932 (Line)
Info    : [

Info    : [ 30%] Meshing curve 5089 (Line)
Info    : [ 30%] Meshing curve 5090 (Line)
Info    : [ 30%] Meshing curve 5091 (Line)
Info    : [ 30%] Meshing curve 5092 (Line)
Info    : [ 30%] Meshing curve 5093 (Line)
Info    : [ 30%] Meshing curve 5094 (Line)
Info    : [ 30%] Meshing curve 5095 (Line)
Info    : [ 30%] Meshing curve 5096 (Line)
Info    : [ 30%] Meshing curve 5097 (Line)
Info    : [ 30%] Meshing curve 5098 (Line)
Info    : [ 30%] Meshing curve 5099 (Line)
Info    : [ 30%] Meshing curve 5100 (Line)
Info    : [ 30%] Meshing curve 5101 (Line)
Info    : [ 30%] Meshing curve 5102 (Line)
Info    : [ 30%] Meshing curve 5103 (Line)
Info    : [ 30%] Meshing curve 5104 (Line)
Info    : [ 30%] Meshing curve 5105 (Line)
Info    : [ 30%] Meshing curve 5106 (Line)
Info    : [ 30%] Meshing curve 5107 (Line)
Info    : [ 30%] Meshing curve 5108 (Line)
Info    : [ 30%] Meshing curve 5109 (Line)
Info    : [ 30%] Meshing curve 5110 (Line)
Info    : [ 30%] Meshing curve 5111 (Line)
Info    : [

...

Info    : [ 90%] Meshing curve 15225 (Line)
Info    : [ 90%] Meshing curve 15226 (Line)
Info    : [ 90%] Meshing curve 15227 (Line)
Info    : [ 90%] Meshing curve 15228 (Line)
Info    : [ 90%] Meshing curve 15229 (Line)
Info    : [ 90%] Meshing curve 15230 (Line)
Info    : [ 90%] Meshing curve 15231 (Line)
Info    : [ 90%] Meshing curve 15232 (Line)
Info    : [ 90%] Meshing curve 15233 (Line)
Info    : [ 90%] Meshing curve 15234 (Line)
Info    : [ 90%] Meshing curve 15235 (Line)
Info    : [ 90%] Meshing curve 15236 (Line)
Info    : [ 90%] Meshing curve 15237 (Line)
Info    : [ 90%] Meshing curve 15238 (Line)
Info    : [ 90%] Meshing curve 15239 (Line)
Info    : [ 90%] Meshing curve 15240 (Line)
Info    : [ 90%] Meshing curve 15241 (Line)
Info    : [ 90%] Meshing curve 15242 (Line)
Info    : [ 90%] Meshing curve 15243 (Line)
Info    : [ 90%] Meshing curve 15244 (Line)
Info    : [ 90%] Meshing curve 15245 (Line)
Info    : [ 90%] Meshing curve 15246 (Line)
Info    : [ 90%] Meshing curve 1

Info    : [ 90%] Meshing curve 15584 (Line)
Info    : [ 90%] Meshing curve 15585 (Line)
Info    : [ 90%] Meshing curve 15586 (Line)
Info    : [ 90%] Meshing curve 15587 (Line)
Info    : [ 90%] Meshing curve 15588 (Line)
Info    : [ 90%] Meshing curve 15589 (Line)
Info    : [ 90%] Meshing curve 15590 (Line)
Info    : [ 90%] Meshing curve 15591 (Line)
Info    : [ 90%] Meshing curve 15592 (Line)
Info    : [ 90%] Meshing curve 15593 (Line)
Info    : [ 90%] Meshing curve 15594 (Line)
Info    : [ 90%] Meshing curve 15595 (Line)
Info    : [ 90%] Meshing curve 15596 (Line)
Info    : [ 90%] Meshing curve 15597 (Line)
Info    : [ 90%] Meshing curve 15598 (Line)
Info    : [ 90%] Meshing curve 15599 (Line)
Info    : [ 90%] Meshing curve 15600 (Line)
Info    : [ 90%] Meshing curve 15601 (Line)
Info    : [ 90%] Meshing curve 15602 (Line)
Info    : [ 90%] Meshing curve 15603 (Line)
Info    : [ 90%] Meshing curve 15604 (Line)
Info    : [ 90%] Meshing curve 15605 (Line)
Info    : [ 90%] Meshing curve 1

Info    : [100%] Meshing curve 17500 (Line)
Info    : [100%] Meshing curve 17501 (Line)
Info    : [100%] Meshing curve 17502 (Line)
Info    : [100%] Meshing curve 17503 (Line)
Info    : [100%] Meshing curve 17504 (Line)
Info    : [100%] Meshing curve 17505 (Line)
Info    : [100%] Meshing curve 17506 (Line)
Info    : [100%] Meshing curve 17507 (Line)
Info    : [100%] Meshing curve 17508 (Line)
Info    : [100%] Meshing curve 17509 (Line)
Info    : [100%] Meshing curve 17510 (Line)
Info    : [100%] Meshing curve 17511 (Line)
Info    : [100%] Meshing curve 17512 (Line)
Info    : [100%] Meshing curve 17513 (Line)
Info    : [100%] Meshing curve 17514 (Line)
Info    : [100%] Meshing curve 17515 (Line)
Info    : [100%] Meshing curve 17516 (Line)
Info    : [100%] Meshing curve 17517 (Line)
Info    : [100%] Meshing curve 17518 (Line)
Info    : [100%] Meshing curve 17519 (Line)
Info    : [100%] Meshing curve 17520 (Line)
Info    : [100%] Meshing curve 17521 (Line)
Info    : [100%] Meshing curve 1

Info    : [100%] Meshing curve 17820 (Line)
Info    : [100%] Meshing curve 17821 (Line)
Info    : [100%] Meshing curve 17822 (Line)
Info    : [100%] Meshing curve 17823 (Line)
Info    : [100%] Meshing curve 17824 (Line)
Info    : [100%] Meshing curve 17825 (Line)
Info    : [100%] Meshing curve 17826 (Line)
Info    : [100%] Meshing curve 17827 (Line)
Info    : [100%] Meshing curve 17828 (Line)
Info    : [100%] Meshing curve 17829 (Line)
Info    : [100%] Meshing curve 17830 (Line)
Info    : [100%] Meshing curve 17831 (Line)
Info    : [100%] Meshing curve 17832 (Line)
Info    : [100%] Meshing curve 17833 (Line)
Info    : [100%] Meshing curve 17834 (Line)
Info    : [100%] Meshing curve 17835 (Line)
Info    : [100%] Meshing curve 17836 (Line)
Info    : [100%] Meshing curve 17837 (Line)
Info    : [100%] Meshing curve 17838 (Line)
Info    : [100%] Meshing curve 17839 (Line)
Info    : [100%] Meshing curve 17840 (Line)
Info    : [100%] Meshing curve 17841 (Line)
Info    : [100%] Meshing curve 1

Info    : [ 20%] Meshing surface 2 (Parametric surface, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 3 (Parametric surface, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 4 (Parametric surface, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 5 (Parametric surface, Frontal-Delaunay)
Info    : [ 80%] Meshing surface 6 (Parametric surface, Frontal-Delaunay)
Info    : [ 90%] Meshing surface 7 (Parametric surface, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.86587s, CPU 0.862364s)
Info    : 33268 nodes 84882 elements


In [17]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.10.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 20220501
Build host    : gmsh.info
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blas[petsc] Blossom Cgns DIntegration Dlopen DomHex Eigen[contrib] Fltk Gmm[contrib] Hxt Jpeg Kbipack Lapack[petsc] LinuxJoystick MathEx[contrib] Med Mesh Metis[contrib] Mmg Mpeg Netgen ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom PETSc Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.4.0
PETSc version : 3.14.4 (real arithmtic)
OCC version   : 7.6.1
MED version   : 4.1.0
Packaged by   : geuzaine
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


In [19]:
gmsh.write("continents.vtk")

Info    : Writing 'continents.vtk'...
Info    : Done writing 'continents.vtk'
